In [1]:
# IMPORTS/REQUIREMENTS
import numpy as np
import pandas as pd
import holidays
import inspect
from scipy.optimize import curve_fit
from tqdm import tqdm
from sklearn.linear_model import LinearRegression
from helper_functions_ea import check_env, ShoojuTools
check_env()
sj = ShoojuTools()

In [2]:
import matplotlib.pyplot as plt

In [3]:
# UTILS
def get_temp_load_curve(data_coef):
    temp_load_curve = data_coef.set_index('temp').sort_index()
    temp_load_curve = temp_load_curve.loc[temp_load_curve.workday == temp_load_curve.workday.max(),'load']
    temp_range = temp_load_curve.index.max() - temp_load_curve.index.min()
    window = int(len(temp_load_curve)/temp_range)
    temp_load_curve = temp_load_curve.rolling(window, center=True).mean().dropna()
    return temp_load_curve

def get_workday(series):
    series_idx = pd.Series(series.index, series.index).asfreq('h')
    weekend = series_idx.dt.dayofweek >= 5
    us_holidays = holidays.US()
    holiday = series_idx.apply(lambda x: x.date() in us_holidays)
    workday = (~holiday & ~weekend).astype(int)
    workday.name = 'workday'
    return workday

def get_func_params(func):
    params = inspect.signature(func).parameters
    params = [param_name for param_name, param in params.items()]
    if 'xdata' in params:
        params.remove('xdata')
    return params

def get_load_sids_dict():
    load_sid_dict = {'ERCOT': fr'teams\power\ERCOT\load\hourly',
                      'PJM': fr'teams\power\PJM\load\hourly'}
    return load_sid_dict

In [4]:
# METADATA
def get_iso_zone_sids(iso_rto, iso_zone):
    # REALISED LOAD
    query = get_load_sids_dict()[iso_rto] + fr'\{iso_zone}'
    load_sid = sj.get_fields_from_query(query, fields=['sid'])['sid']
    load_sid.index = load_sid.str[len(query)-5:]
    load_sid.name = 'load_real'

    #EA GROWTH FORECAST
    query = fr'users\will.spencer\power\models\load\growth\total\{iso_rto}\{iso_zone}'
    growth_sid = sj.get_fields_from_query(query, fields=['sid'])['sid']
    growth_sid.index = growth_sid.str[len(query)-5:]
    growth_sid.name = 'growth_total'

    # REALISED HDD'S AND CDD'S
    if iso_zone == iso_rto:
        query = fr'sid:teams\weather\spire\us_POWER_{iso_rto} unit=C'
    else:
        query = fr'sid:teams\weather\spire\us_POWER_{iso_rto}_{iso_zone} unit=C'
    real_degree_sid = sj.get_fields_from_query(query, fields=['sid'])['sid']
    real_degree_sid.index = real_degree_sid.str[len(query)-16:-4].replace('', iso_rto)
    hdd_real_sid = real_degree_sid.loc[real_degree_sid.str.lower().str.contains('hdd')]
    hdd_real_sid.name = 'hdd_real'
    cdd_real_sid = real_degree_sid.loc[real_degree_sid.str.lower().str.contains('cdd')]
    cdd_real_sid.name = 'cdd_real'

    # NORMAL HDD'S AND CDD'S
    if iso_zone == iso_rto:
        query = fr'sid:Spire\weather\10yr_normal\NA\US\POWER_{iso_rto} unit=C'
    else:
        query = fr'sid:Spire\weather\10yr_normal\NA\US\POWER_{iso_rto}_{iso_zone} unit=C'

    norm_degree_sid = sj.get_fields_from_query(query, fields=['sid'])['sid']
    norm_degree_sid.index = norm_degree_sid.str[len(query)-16:-6].replace('', iso_rto)
    hdd_norm_sid = norm_degree_sid.loc[norm_degree_sid.str.lower().str.contains('hdd')]
    hdd_norm_sid.name = 'hdd_norm'
    cdd_norm_sid = norm_degree_sid.loc[norm_degree_sid.str.lower().str.contains('cdd')]
    cdd_norm_sid.name = 'cdd_norm'

    # CONCAT AND CHECK ALL DATA EXISTS
    iso_zone_sids = pd.concat([load_sid, growth_sid, hdd_real_sid, cdd_real_sid, hdd_norm_sid, cdd_norm_sid], axis=1).dropna()
    if len(iso_zone_sids)>0:
        iso_zone_sids = iso_zone_sids.iloc[0]
        sids_exist = True
    else:
        sids_exist = False

    return sids_exist, iso_zone_sids

In [5]:
# EXTRACTORS
def get_temp_real(iso_zone_sids):
    cdd_real = sj.get_points_from_sid_into_series(iso_zone_sids.cdd_real, date_start="MIN", date_finish="MAX")
    hdd_real = sj.get_points_from_sid_into_series(iso_zone_sids.hdd_real, date_start="MIN", date_finish="MAX")
    temp_real = cdd_real-hdd_real+18
    temp_real = temp_real.asfreq('H').ffill()
    temp_real.name = 'temp'
    return temp_real

def get_temp_norm(iso_zone_sids, data):
    cdd_norm = sj.get_points_from_sid_into_series(iso_zone_sids.cdd_norm, date_start="MIN", date_finish="MAX")
    hdd_norm = sj.get_points_from_sid_into_series(iso_zone_sids.hdd_norm, date_start="MIN", date_finish="MAX")
    temp_norm = cdd_norm-hdd_norm+18
    temp_norm.index = temp_norm.index.dayofyear
    temp_norm = temp_norm.groupby(level=0).agg(pd.Series.mode)
    
    norm_start = data['temp_real'].index.min()
    norm_end = data['growth_total'].index.max() + pd.offsets.YearEnd()
    norm_date_range = pd.date_range(norm_start, norm_end, freq='H')

    temp_norm = norm_date_range.day_of_year.map(temp_norm)
    temp_norm = pd.Series(temp_norm, norm_date_range)
    temp_norm.name = 'temp'
    return temp_norm

def get_load_real(iso_zone_sids):
    load_real = sj.get_points_from_sid_into_series(iso_zone_sids.load_real, date_start="MIN", date_finish="MAX")
    load_real.name = 'load'
    return load_real

def get_growth_total(iso_zone_sids):
    growth_total = sj.get_points_from_sid_into_series(iso_zone_sids.growth_total, date_start="MIN", date_finish="MAX")
    growth_total.name = 'growth_total'
    return growth_total

def get_data(iso_zone_sids):
    data = {'temp_real':get_temp_real(iso_zone_sids),
            'load_real':get_load_real(iso_zone_sids),
            'growth_total': get_growth_total(iso_zone_sids)}
    data['temp_norm'] = get_temp_norm(iso_zone_sids, data)
    return data


In [6]:
# MODEL DATA

# rolling (DAILY) MODEL
def get_data_coef(data):
    data_coef = pd.concat([data['load_real'], data['temp_real']], axis=1).dropna()
    data_coef['workday'] = get_workday(data_coef)
    data_coef = data_coef.asfreq('H').rolling(24, center=True).mean().dropna()
    return data_coef

In [7]:
# LOAD MODEL FUNCS

def base_load_coefs_func(xdata, base_demand, workday_demand):
    base_load = base_demand + workday_demand*xdata.workday
    return base_load

def heat_load_coefs_func(xdata, util_temp_max, heat_util_mask, heat_demand):
    heat_load = (heat_demand)*(util_temp_max-xdata.temp)*heat_util_mask
    return heat_load

def cool_load_coefs_func(xdata, util_temp_min, cool_util_mask, cool_demand):
    cool_load = (cool_demand)*(xdata.temp-util_temp_min)*cool_util_mask
    return cool_load

def util_func(xdata, util_temp_min, util_temp_max):
    heat_util_mask = (xdata.temp - util_temp_max)/(util_temp_min-util_temp_max)
    heat_util_mask.loc[xdata.temp<util_temp_min] = 1
    heat_util_mask.loc[xdata.temp>util_temp_max] = 0

    cool_util_mask = (util_temp_min-xdata.temp)/(util_temp_min-util_temp_max)
    cool_util_mask.loc[xdata.temp>util_temp_max] = 1
    cool_util_mask.loc[xdata.temp<util_temp_min] = 0

    return heat_util_mask, cool_util_mask

def load_coefs_func(xdata, base_demand, workday_demand, heat_demand, cool_demand, util_temp_min, util_temp_max):

    base_load = base_load_coefs_func(xdata, base_demand, workday_demand)
    heat_util_mask, cool_util_mask = util_func(xdata, util_temp_min, util_temp_max)

    heat_load = heat_load_coefs_func(xdata, util_temp_max, heat_util_mask, heat_demand)
    cool_load = cool_load_coefs_func(xdata, util_temp_min, cool_util_mask, cool_demand)

    load = base_load + heat_load + cool_load
    return load

In [8]:
# GET DATE TO FIT INITIAL LOAD MODEL COEFS
def get_coef_init_date(data_coef):
    temp_load_min = get_temp_load_curve(data_coef).idxmin()

    heat_start_date = (data_coef.temp > temp_load_min).groupby(pd.Grouper(freq='W')).mean()
    heat_start_date = heat_start_date[heat_start_date == 1].index.min()

    cool_start_date = (data_coef.temp < temp_load_min).groupby(pd.Grouper(freq='W')).mean()
    cool_start_date = cool_start_date[cool_start_date == 1].index.min()

    coef_init_date = max(heat_start_date, cool_start_date)
    coef_init_date = coef_init_date + pd.offsets.MonthBegin(2)
    return coef_init_date

# INITIAL bounding
def get_init_bounds(coef_init_data):
    init_curve = get_temp_load_curve(coef_init_data)
    temp_min = init_curve.index.min()
    temp_max = init_curve.index.max()
    curve_min = init_curve.idxmin()
    load_min = init_curve.min()

    init_bounds = {'base_demand':{'lower': 0,
                                  'upper': load_min},
                   'workday_demand':{'lower': 0,
                                  'upper': load_min},
                   'heat_demand':{'lower': 0,
                                  'upper': np.inf},
                   'cool_demand':{'lower': 0,
                                  'upper': np.inf},
                   'util_temp_min':{'lower': temp_min,
                                    'upper': curve_min},
                   'util_temp_max':{'lower': curve_min,
                                    'upper': temp_max}}

    init_bounds = pd.DataFrame(init_bounds).transpose()
    init_bounds = (init_bounds.lower.to_list(), init_bounds.upper.to_list())
    return init_bounds

# INITIAL COEF PARAMETERS
def fit_start_params(data_coef, init_date, init_bounds):
    xdata = data_coef.loc[:init_date, data_coef.columns.drop('load')]
    ydata = data_coef.loc[:init_date, 'load']
    popt, pcov = curve_fit(load_coefs_func, xdata, ydata, bounds=init_bounds)
    perr = np.sqrt(np.diag(pcov))

    params = get_func_params(load_coefs_func)
    start_params = pd.DataFrame([popt-3*perr, popt, popt+3*perr], ['lower','coef','upper'], params)
    start_params = start_params.transpose()
    return start_params

# BOOTSTRAP DATA FOR GIVEN DATE
def bootstrap_fit_data(data, fit_date, params):
    trailing = data.loc[fit_date - pd.DateOffset(28):fit_date]
    trailing = trailing.loc[trailing.load > params.lower.base_demand]
    leading = data.loc[fit_date:fit_date + pd.DateOffset(28)]
    leading = leading.loc[leading.load > params.lower.base_demand]

    if len(trailing)>0 and len(leading) > 0:
        trailing = trailing.iloc[np.random.randint(0, len(trailing), 1000)]
        leading = leading.iloc[np.random.randint(0, len(leading), 1000)]
        fit_data = pd.concat([trailing, leading]).reset_index(drop=True)
        
    else:
        fit_data = pd.DataFrame()
    
    return fit_data


In [9]:
# FIT WEEKLY DATA
def fit_load_wrap(fit_data, wrap, params, scale_params):
    xdata = fit_data.drop(columns='load')
    ydata = fit_data.load

    fit_params = get_func_params(wrap)
    if len(scale_params) > 0: params.loc['scale'] = [-np.inf,1,np.inf]

    lower = params.loc[fit_params, 'lower'].to_list()
    p0 = params.loc[fit_params, 'coef'].to_list()
    upper = params.loc[fit_params, 'upper'].to_list()
    bounds = (lower, upper)

    popt, pcov = curve_fit(wrap, xdata, ydata, p0=p0, bounds=bounds)
    perr = np.sqrt(np.diag(pcov))
    params.loc[fit_params, 'lower'] = popt - 3*perr
    params.loc[fit_params, 'coef'] = popt
    params.loc[fit_params, 'upper'] = popt + 3*perr

    if len(scale_params) > 0: 
        params.loc[scale_params] = params.loc[scale_params]*params.coef.scale
        params = params.drop('scale')

    return params

# WEEKLY FIT TYPES
# HEAT
def fit_heat_params(fit_data, params):
    def load_wrap(xdata, base_demand, workday_demand, heat_demand, util_temp_min):
        load = load_coefs_func(xdata, 
                        base_demand, workday_demand, 
                        heat_demand, params.coef.cool_demand, 
                        util_temp_min, params.coef.util_temp_max)
        return load
    scale_params = []
    params = fit_load_wrap(fit_data, load_wrap, params, scale_params)

    return params

def fit_cool_params(fit_data, params):
    def load_wrap(xdata, base_demand, workday_demand, cool_demand, util_temp_max):
        load = load_coefs_func(xdata, 
                        base_demand, workday_demand, 
                        params.coef.heat_demand, cool_demand, 
                        params.coef.util_temp_min, util_temp_max)
        return load
    scale_params = []
    params = fit_load_wrap(fit_data, load_wrap, params, scale_params)

    return params

def fit_base_params(fit_data, params):
    def load_wrap(xdata, base_demand, workday_demand, scale):
        load = load_coefs_func(xdata, 
                        base_demand, workday_demand, 
                        scale*params.coef.heat_demand, scale*params.coef.cool_demand, 
                        params.coef.util_temp_min, params.coef.util_temp_max)
        return load
    scale_params = ['heat_demand', 'cool_demand']
    params = fit_load_wrap(fit_data, load_wrap, params, scale_params)

    return params

def scale_params(fit_data, params):
    def load_wrap(xdata, scale):
        load = load_coefs_func(xdata, 
                        scale*params.coef.base_demand, scale*params.coef.workday_demand, 
                        scale*params.coef.heat_demand, scale*params.coef.cool_demand, 
                        params.coef.util_temp_min, params.coef.util_temp_max)
        return load
    scale_params = ['base_demand', 'workday_demand', 'heat_demand', 'cool_demand']
    params = fit_load_wrap(fit_data, load_wrap, params, scale_params)

    return params

In [10]:
# DETERMINE WHICH PART OF CURVE SHOULD BE FIT AND FIT IT
def fit_date_coefs(fit_data, params):
    if len(fit_data)>0:

        heat_mask = fit_data.temp <= params.coef.util_temp_min
        cool_mask = fit_data.temp >= params.coef.util_temp_max

        fit_heat = heat_mask.mean() > 0.5
        fit_cool = cool_mask.mean() > 0.5
        fit_util = heat_mask.sum() > 0 and cool_mask.sum() > 0       
    else:
        return params
    
    if fit_heat and fit_util: return fit_heat_params(fit_data, params)
    elif fit_cool and fit_util: return fit_cool_params(fit_data, params)
    elif fit_util: return fit_base_params(fit_data, params)
    else: return scale_params(fit_data, params)



In [11]:
def fit_coefs(data):
    coefs = dict()
    data_coef = get_data_coef(data)
    init_date = get_coef_init_date(data_coef)
    init_bounds = get_init_bounds(data_coef.loc[:init_date])
    params = fit_start_params(data_coef, init_date, init_bounds)

    coef_date_range = pd.date_range(init_date, data_coef.index.max(), freq='W')
    for fit_date in (coef_date_range):
        fit_data = bootstrap_fit_data(data_coef, fit_date, params)
        params = fit_date_coefs(fit_data, params)
        coefs[fit_date] = params.coef.copy()

    coefs = pd.DataFrame(coefs).transpose()
    return coefs


In [35]:
def get_xdata_coef(temp):
    xdata = pd.DataFrame(temp)
    xdata['workday'] = get_workday(xdata)
    xdata = xdata.asfreq('H').rolling(24, center=True).mean().dropna()
    return xdata

def get_load_coefs(temp, coefs):
    load_coefs = pd.Series(dtype='float')
    xdata_coef = get_xdata_coef(temp)
    coef_date_range = coefs.index
    for pred_date in coef_date_range:
        xdata = xdata_coef.loc[pred_date-pd.DateOffset(28):pred_date+pd.DateOffset(28)]
        load_coefs = pd.concat([load_coefs, load_coefs_func(xdata, *coefs.loc[pred_date])])
    load_coefs = load_coefs.groupby(level=0).mean()
    load_coefs.name = 'load'

    return load_coefs

def get_data_hour(data, coefs):
    load_pred = get_load_coefs(data['temp_real'], coefs)
    load_real = data['load_real'].copy()
    load_err = pd.Series(load_pred/load_real, name = 'load_err')
    data_hour = pd.concat([load_real, load_err], axis=1).dropna()
    
    return data_hour

def fit_hour_adj(data, coefs):
    hour_adj = dict()
    data_hour = get_data_hour(data, coefs)
    coef_date_range = coefs.index
    for fit_date in coef_date_range:
        fit_data = data_hour.loc[fit_date-pd.DateOffset(28):fit_date+pd.DateOffset(28)].dropna()
        load_loss = coefs.loc[fit_date,'base_demand'] - 3*fit_data.load.std()
        fit_data = fit_data.loc[fit_data.load > load_loss, 'load_err']
        hour_adj[fit_date] = fit_data.groupby(fit_data.index.hour).mean()

    hour_adj = pd.DataFrame(hour_adj).transpose()
    return hour_adj

In [90]:
def run_load_model(temp, coefs, hour_adj):
    xdata_coef = get_xdata_coef(temp)
    load = pd.Series(dtype='float')
    pred_date_range = pd.concat([coefs, hour_adj], axis=1).index
    for pred_date in pred_date_range:
        pred_data = xdata_coef.loc[pred_date-pd.DateOffset(28):pred_date+pd.DateOffset(28)].dropna()
        pred_date_load = load_coefs_func(pred_data, *coefs.loc[pred_date])
        pred_date_load = pred_date_load*pred_date_load.index.hour.map(hour_adj.loc[pred_date])
        load = pd.concat([load, pred_date_load])
    load = load.groupby(level=0).mean()

    return load

In [313]:
def get_forecast_growth_adj(data, date_range):
    growth_rate = (1 + data['growth_total'])**(1/8760)
    growth_rate = growth_rate.reindex(date_range).bfill().ffill()

    forecast_growth_adj = dict()

    growth_factor = 1
    for growth_date in date_range:
        growth_factor = growth_factor*growth_rate.loc[growth_date]
        forecast_growth_adj[growth_date] = growth_factor

    forecast_growth_adj = pd.Series(forecast_growth_adj)
    return forecast_growth_adj

def get_forecast_hour_adj(hour_adj, date_range):
    doy_hour_adj = hour_adj.loc[hour_adj.index.max() - pd.DateOffset(365):]
    doy_hour_adj = doy_hour_adj.asfreq('D').ffill()
    doy_hour_adj = doy_hour_adj.groupby(doy_hour_adj.index.dayofyear).mean()
    doy_hour_adj.loc[366] = doy_hour_adj.loc[365]

    forecast_hour_adj = dict()

    for hour_date in date_range:
        forecast_hour_adj[hour_date] = doy_hour_adj.loc[hour_date.dayofyear, hour_date.hour]

    forecast_hour_adj = pd.Series(forecast_hour_adj)

    return forecast_hour_adj

def get_load_norm(data, coefs, hour_adj):
    load_norm = run_load_model(data['temp_norm'], coefs, hour_adj)
    forecast_data = data['temp_norm'].loc[load_norm.index.max():]
    forecast_data = pd.concat([forecast_data, get_workday(forecast_data)], axis=1)

    forecast_load = load_coefs_func(forecast_data, *coefs.loc[coefs.index.max()])
    forecast_date_range = forecast_load.index

    forecast_growth_adj = get_forecast_growth_adj(data, forecast_date_range)
    forecast_hour_adj = get_forecast_hour_adj(hour_adj, forecast_date_range)
    forecast_load = forecast_load*forecast_growth_adj*forecast_hour_adj

    load_norm = pd.concat([load_norm, forecast_load])
    load_norm.name = 'load'
    return load_norm

In [299]:
iso_rto = 'ERCOT'
iso_zone = 'ERCOT'

sids_exist, iso_zone_sids = get_iso_zone_sids(iso_rto, iso_zone)
if sids_exist:
    data = get_data(iso_zone_sids)
    coefs = fit_coefs(data)
    hour_adj = fit_hour_adj(data, coefs)
    load_norm = get_load_norm(data, coefs, hour_adj)


In [314]:
load_norm = get_load_norm(data, coefs, hour_adj)

In [317]:
load_norm

2013-06-09 00:00:00    42521.740618
2013-06-09 01:00:00    46188.546506
2013-06-09 02:00:00    49261.182974
2013-06-09 03:00:00    51687.524236
2013-06-09 04:00:00    53362.967892
                           ...     
2030-12-30 20:00:00    54773.422551
2030-12-30 21:00:00    55416.339897
2030-12-30 22:00:00    56035.543794
2030-12-30 23:00:00    55898.850289
2030-12-31 00:00:00    54924.840661
Name: load, Length: 153938, dtype: float64